In [1]:
# Import paakage
using Distributions, Random, DelimitedFiles, Shell, LinearAlgebra, Glob, ZipFile, Interpolations

In [ ]:
# udOrientationTensorGenerator
# Randomly sample a uni-directional random reference orientation tensor.
# Randomly sample a reference fiber volume fraction from a given value +0-50%.
#
# fibervf:          Base volume fraction of fiber in the composite material.
# path:             File path including file name and number (will append .txt).
# path2:            File path for RVE size, including file name and number (will append .txt).
#
function udOrientationTensorGenerator(fibervf,path,path2="")
    tmp = rand(1:3)
    if tmp==1
        a = [1 0 0; 0 0 0; 0 0 0]
        sizeRVE = [9.6e-4 6e-5 6e-5]
    elseif tmp==2
        a = [0 0 0; 0 1 0; 0 0 0]
        sizeRVE = [6e-5 9.6e-4 6e-5]
    elseif tmp==3
        a = [0 0 0; 0 0 0; 0 0 1]
        sizeRVE = [6e-5 6e-5 9.6e-4]
    end

    # Randomly sample a volume fraction between initial volume fraction + 0-50%
    vf = string(parse(Float64,fibervf) + parse(Float64,fibervf)*0.5*rand())
    # Write to file!
    header = ["a11" "a22" "a33" "a12" "a13" "a23" "vf"]
    currentPath = path*".txt"
    open(currentPath, "w") do io
        writedlm(io,vcat(header,hcat(a[1,1],a[2,2],a[3,3],a[1,2],a[1,3],a[2,3],
        vf)))
    end
    header = ["rve_size_x" "rve_size_y" "rve_size_z"]
    currentPath = path2*".txt"
    open(currentPath, "w") do io
        writedlm(io,vcat(header,hcat(sizeRVE[1],sizeRVE[2],sizeRVE[3])))
    end
    return nothing
end

In [ ]:
# strainPathGenerator
# Generates a path in 6D strain space and prints to a tab delimited file.
# Picks n_drift directions and walks n_step steps in those directions with
# added noise. The noise vector is multiplied by the variable noise.
# The directions and noise is sampled from a normal distribution with mean
# 0 and standard deviation 1. The output data file is formatted column wise
# given in the order "time" "e11" "e22" "e33" "2*e12" "2*e23" "2*e13".
#
# noise:    Noise multiplicative factor
# n_step:   Number of random steps per directions
# n_drift:  Number of drift directions
# EPS:      Maximum admissible strain component
# t:        Time variable
# path:     File path including file name and number (will append .txt).
# uni:      If true; sets all but one random strain component to 0.

function strainPathGenerator(noise,n_step,n_drift,EPS,t,path,uni)
    X = repeat(rand(Normal(0,1),(n_drift,6)),inner=[n_step, 1])
    X = X./sqrt.(sum((X).^2,dims=2))

    Y = rand(Normal(0,1),(n_drift*n_step,6))
    Y = noise.*Y./sqrt.(sum((Y).^2,dims=2))

    Eps = vcat(zeros(1,6),cumsum(X+Y,dims=1))

    # For unidirectional strain, delete all components except for one.
    if uni
        zeroIndex = [1;2;3;4;5;6]
        deleteat!(zeroIndex,rand(1:6))
        Eps[:,zeroIndex] .= 0
    end

    s = maximum(abs.(Eps))
    eps = Eps.*EPS./s
    # Input format is shearstrain*2, therefore those components are scaled.
    eps[:,4:end] = eps[:,4:end]*2

    header = ["time" "e11" "e22" "e33" "2*e12" "2*e23" "2*e13"]
    currentPath =path*".txt"
    open(currentPath, "w") do io
        writedlm(io, vcat(header,hcat(t,eps)))
    end

    return nothing
end

In [ ]:
# writedaf
# Writes *.mat for analysis of linear exponential isotropic hardening matrix
# with elastic fibers. The loading conditions are loaded from a strain path
# file. The loading conditions are general 3D loading, prescribed 6D strain.
#
# inPath:                      The path of the input strain file.
# inPath2:                     The path of the input orientation file.
# inPath3:                     The path of the RVE size file.
# outPath:                     The path of the output *.daf file.
# finalTime:                   Final time of the analysis.
# maxTimeInc:                  Maximum integration time step.
# minTimeInc:                  Minimum integration time step.
# initialTimeInc:              Initial time inctement.
# youngFiber:                  Young's modulus of fiber.
# poissonFiber:                Poisson's ratio of fiber.
# fiberDiameter:          	   Fiber diameter.
# fiberSize:				   Fiber size/ length.
# fiberOrientation:            Fiber orientation.
# youngMatrix:                 Young's modulus of matrix.
# poissonMatrix:               Poisson's ratio of matrix.
# yieldMatrix:                 Yield stress of matrix.
# hardeningModulusMatrix:      Hardening modulus R_inf of matrix.
# hardeningExponentMatrix:     Hardening exponent m of matrix.
# hardeningModulus2Matrix:     Linear hardening modulus of matrix.
#
function writedaf(inPath,inPath2,inPath3,outPath,finalTime,maxTimeInc,minTimeInc,initialTimeInc,
    youngFiber,poissonFiber,fiberDiameter,fiberSize,fiberOrientation,
    youngMatrix,poissonMatrix,yieldMatrix,hardeningModulusMatrix,
    hardeningExponentMatrix,hardeningModulus2Matrix)

    analysisName = outPath[findlast("/",outPath)[1]+1:end-4]

    separator = "##########################################"

    # Begin by reading strain data file.
    strainData = open(inPath, "r") do io
        readdlm(io, '\t')
    end

    # Read orientation tensor if applicable.
    orientationTensor = open(inPath2, "r") do io
        readdlm(io, '\t')
    end
    fiberVolumefraction = string.(orientationTensor[end])
    matrixVolumefraction = string.(1-orientationTensor[end])
    a = string.(orientationTensor[2,:])
    
    sizeRVE = readdlm(inPath3,skipstart=1)
    sizeRVEx = string(sizeRVE[1])
    sizeRVEy = string(sizeRVE[2])
    sizeRVEz = string(sizeRVE[3])

    # Define entries for sections of *.mat
    fiberMATERIAL = [""; separator; "MATERIAL"; "name = Fiber"; "type = elastic";
    "elastic_model = isotropic"; "Young = "*youngFiber; "Poisson = "*poissonFiber; ""]

    matrixMATERIAL = [separator; "MATERIAL"; "name = Matrix";
    "type = J2_plasticity"; "consistent_tangent = on"; "elastic_model = isotropic";
    "Young = "*youngMatrix; "Poisson = "*poissonMatrix;
    "yield_stress = "*yieldMatrix; "hardening_model = exponential_linear";
    "hardening_modulus = "*hardeningModulusMatrix;
    "hardening_exponent = "*hardeningExponentMatrix;
    "hardening_modulus2 = "*hardeningModulus2Matrix; "isotropic_method = spectral"; ""]

    fiberPHASE = [separator; "PHASE";"name = InclusionPhase";
    "type = inclusion_fe"; "volume_fraction = "*fiberVolumefraction;
    "material = Fiber"; "inclusion_shape = cylinder";
    "phase_definition = by_size_and_diameter";
    "inclusion_diameter = "*fiberDiameter; "inclusion_size = "*fiberSize;
    "size_distribution = fixed";
    "orientation = "*fiberOrientation; 
    "orientation_11 = "*a[1]; "orientation_22 = "*a[2];
    "orientation_33 = "*a[3]; "orientation_12 = "*a[4];
    "orientation_13 = "*a[5]; "orientation_23 = "*a[6];
    "coated = no"; "interface_behavior = perfectly_bonded"; 
    "clustering = no"; "allow_size_reduction = no"; 
    "track_percolation_onset = no"; "stop_at_percolation = no"; 
    "check_final_percolation = no"; "no_tie_on_fiber_tips = no"; ""]

    matrixPHASE = [separator; "PHASE";"name = MatrixPhase"; "type = matrix";
    "volume_fraction = "*matrixVolumefraction; "material = Matrix"; ""]

    MICROSTRUCTURE = [separator; "MICROSTRUCTURE"; "name = TheMicrostructure";
    "phase = MatrixPhase"; "phase = InclusionPhase"; ""]

    LOADING = [separator; "LOADING"; "name = Mechanical"; "type = strain";
    "load = General_3D"; "initial_strain_11 = 0.0e+00"; "strain_11 = 1.0e+00";
    "initial_strain_22 = 0.0e+00"; "strain_22 = 1.0e+00";
    "initial_strain_33 = 0.0e+00"; "strain_33 = 1.0e+00";
    "initial_strain_12 = 0.0e+00"; "strain_12 = 1.0e+00";
    "initial_strain_23 = 0.0e+00"; "strain_23 = 1.0e+00";
    "initial_strain_13 = 0.0e+00"; "strain_13 = 1.0e+00";
    "history = user_defined"; "history_component_11 = e11";
    "history_component_11_value = relative";
    "history_component_22 = e22"; "history_component_22_value = relative";
    "history_component_33 = e33"; "history_component_33_value = relative";
    "history_component_12 = e12"; "history_component_12_value = relative";
    "history_component_23 = e23"; "history_component_23_value = relative";
    "history_component_13 = e13"; "history_component_13_value = relative";
    "quasi_static = on"; ""]

    RVE = [separator; "RVE"; "type = classical"; "microstructure = TheMicrostructure"; ""]

    MESH = [separator; "MESH"; "mesh_type = conforming"; "automatic_mesh_sizing = on"; 
    "element_size = 4.0e-06"; "minimum_element_size = 5.0e-07"; 
    "use_quadratic_elements = on"; "use_quadratic_geometric_elements = on"; 
    "element_shape = quad_dominated"; "internal_coarsening = on"; "curvature_control = on"; 
    "chordal_deviation_ratio = 1.00e-01"; "nb_refinement_steps = 5";
    "model_layer_interfaces = off"; "seed_size = 2.5e-05"; "share_nodes = off";
    "periodic_mesh = off"; "cohesive_element_size_ratio = 2.0e-01"; ""]

    ANALYSISFE = [separator; "ANALYSISFE"; "name = "*analysisName; "type = mechanical";
    "loading_name = Mechanical";
    "final_time = "*finalTime; "max_time_inc = "*maxTimeInc;
    "min_time_inc = "*minTimeInc; "finite_strain = off";
    "initial_time_inc = "*initialTimeInc; "max_number_increment = 3000";
    "rve_size_definition = user_defined"; "rve_dimension = 3d";
    "size_rve_x = "*sizeRVEx; "size_rve_y = "*sizeRVEy; "size_rve_z = "*sizeRVEz;
    "periodic = yes"; "generation_sequence = proportional"; "generate_matrix = no";
    "track_global_percolation_onset = no"; "stop_at_global_percolation = no";
    "check_final_global_percolation = no"; "random_seed_type = automatic";
    "random_seed = -121641029"; "fe_solver = DigimatFESolver"; 
    "default_timestepping = no"; "nb_cpus = 24"; "fe_solver_type = casi_iterative";
    "segment_to_segment_contact = no"; "linearized_contact = no"; 
    "node_projection = no"; "fe_iterative_solver_tolerance = 1.0e-08";
    "fe_field_output_frequency = 1"; ""]

    GLOBAL_SETTINGS = [separator; "GLOBAL_SETTINGS"; "allow_interpenetration = no"; 
    "allow_coating_interpenetration = no"; "allow_rim_interpenetration = no"; 
    "use_median_plane_interpenetration = no"; "cubic_architecture = no"; 
    "apply_perturbation = no"; "favor_orientation_over_fraction = no";
    "minimum_relative_distance_wrt_diameter = 5.0e-02"; "minimum_relative_vol = 5.0e-02"; 
    "max_number_of_tests = 100000"; "OT_norm_tol = 1.0e-01"; "max_number_of_geometry_attempts = 10";
    "minimum_rel_dist_incl_to_face = 0.0e+00"; "maximum_interpenetration_amount = 1.0e+00";
    "random_fiber_perturbation_no_transverse_perturbation = no"; "default_geometric_options = no";
    "remove_unconnected_matrix_regions = no"; ""]

    point = repeat(["point = "],outer=length(strainData[2:end,1]))

    e11FUNCTION = [separator; "FUNCTION"; "name = e11"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,2]); ""]

    e22FUNCTION = [separator; "FUNCTION"; "name = e22"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,3]); ""]

    e33FUNCTION = [separator; "FUNCTION"; "name = e33"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,4]); ""]

    e12FUNCTION = [separator; "FUNCTION"; "name = e12"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,5]); ""]

    e23FUNCTION = [separator; "FUNCTION"; "name = e23"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,6]); ""]

    e13FUNCTION = [separator; "FUNCTION"; "name = e13"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,7])]

    open(outPath, "w") do io
        writedlm(io,fiberMATERIAL)
        writedlm(io,matrixMATERIAL)
        writedlm(io,matrixPHASE)
        writedlm(io,fiberPHASE)
        writedlm(io,MICROSTRUCTURE)
        writedlm(io,LOADING)
        writedlm(io,RVE)
        writedlm(io,MESH)
        writedlm(io,ANALYSISFE)
        writedlm(io,GLOBAL_SETTINGS)
        writedlm(io,e11FUNCTION)
        writedlm(io,e22FUNCTION)
        writedlm(io,e33FUNCTION)
        writedlm(io,e12FUNCTION)
        writedlm(io,e23FUNCTION)
        writedlm(io,e13FUNCTION)
    end
end

In [ ]:
function unzip(file,exdir="")
    fileFullPath = isabspath(file) ?  file : joinpath(pwd(),file)
    basePath = dirname(fileFullPath)
    outPath = (exdir == "" ? basePath : (isabspath(exdir) ? exdir : joinpath(pwd(),exdir)))
    isdir(outPath) ? "" : mkdir(outPath)
    zarchive = ZipFile.Reader(fileFullPath)
    for f in zarchive.files
        fullFilePath = joinpath(outPath,f.name)
        if (endswith(f.name,"/") || endswith(f.name,"\\"))
            mkdir(fullFilePath)
        else
            write(fullFilePath, read(f))
        end
    end
    close(zarchive)
end

In [ ]:
# extracrResult
# Extract stress strain data to designated result folder if the simultion was completed
#
# inPath: Location of the folder storing Digimat PostPro zip file
# outPath: Location of the folder to store stress-strain data
# dafNameFormat: daf file name format
# outputFileNameFormat: Digimat job name format
# stressStrainResultNameFormat: How to name the stress-strain data file (will append simulation number and .txt)
#
function extracrResult(inPath,outPath,dafNameFormat,outputFileNameFormat,stressStrainResultNameFormat)
    
    listOfSubmittedAnalysisNum = []
    # Unzip pre-process and post-process zip file from Digimat
    digimatPostorcessFile = glob(dafNameFormat*"*.zip",inPath)
    for i = 1:length(digimatPostorcessFile)
        currentPostProcessFile = digimatPostorcessFile[i]
        currentAnalysisNum = split(currentPostProcessFile, "_")[end-3]
        currentPreProcessFile = glob(outputFileNameFormat*dafNameFormat*currentAnalysisNum*".zip",inPath)
        if currentPreProcessFile != []
            listOfSubmittedAnalysisNum = push!(listOfSubmittedAnalysisNum, currentAnalysisNum)
            unzip(currentPostProcessFile,inPath)
            unzip(currentPreProcessFile[1],inPath)
        end
        # Clear used variable
        currentAnalysisNum=[]
        currentPreProcessFile=[]
        currentPostProcessFile=[]
    end

    # Extract stress strain data, calculate actual fiber VF and orientation tensor
    listOfIncompleteAnalysisNum = []
    listOfAnalysisActualFinalTime = []
    for i = 1:length(listOfSubmittedAnalysisNum)
        unzippedMacFile = glob(outputFileNameFormat*dafNameFormat*listOfSubmittedAnalysisNum[i]*".mac",inPath)
        unzippedInputFile = glob(outputFileNameFormat*dafNameFormat*listOfSubmittedAnalysisNum[i]*".daf",inPath)
        unzippedOrientationFile = glob(outputFileNameFormat*dafNameFormat*listOfSubmittedAnalysisNum[i]*"_InclusionPhase_orientation.txt",inPath)
        unzippedFiberVolumeFile = glob(outputFileNameFormat*dafNameFormat*listOfSubmittedAnalysisNum[i]*"_InclusionPhase_volumeIncl.txt",inPath)
        
        input = readdlm(unzippedInputFile[1])
        result = readdlm(unzippedMacFile[1],skipstart=2)
        refFinalTime = input[findlast(x->x=="final_time",input)[1],3]
        actualFinalTime = result[end,1]
        listOfAnalysisActualFinalTime = push!(listOfAnalysisActualFinalTime, ": "*string(actualFinalTime))

        # Only extract and calculate data if the analysis was completed
        if actualFinalTime == refFinalTime
            # Open fiber orientation file
            actualFiberOrientation = readdlm(unzippedOrientationFile[1],';',skipstart=1)
            uniqueFiberNum = findfirst.(isequal.(unique(actualFiberOrientation[:,1])), [actualFiberOrientation[:,1]])
            actualFiberOrientation = actualFiberOrientation[uniqueFiberNum,:]

            # Calculate actual orientation tensor
            theta = actualFiberOrientation[:,3]
            phi = actualFiberOrientation[:,4]
            p = [sind.(theta).*cosd.(phi) sind.(theta).*sind.(phi) cosd.(theta)]
            actualOrientationTensor = zeros(1,6)
            actualOrientationTensor[1] = sum(p[:,1].*p[:,1])
            actualOrientationTensor[2] = sum(p[:,2].*p[:,2])
            actualOrientationTensor[3] = sum(p[:,3].*p[:,3])
            actualOrientationTensor[4] = sum(p[:,1].*p[:,2])
            actualOrientationTensor[5] = sum(p[:,1].*p[:,3])
            actualOrientationTensor[6] = sum(p[:,2].*p[:,3])
            actualOrientationTensor = actualOrientationTensor./sum(actualOrientationTensor[1:3]) 

            # Calculate actual fiber VF
            actualFiberVolume = readdlm(unzippedFiberVolumeFile[1],';',skipstart=1)
            actualFiberVolume = sum(actualFiberVolume[:,2])
            if "size_rve" in input
                sizeRVEx = input[findlast(x->x=="size_rve",input)[1],3]
                sizeRVEy = sizeRVEx
                sizeRVEz = sizeRVEx
            else
                sizeRVEx = input[findlast(x->x=="size_rve_x",input)[1],3]
                sizeRVEy = input[findlast(x->x=="size_rve_y",input)[1],3]
                sizeRVEz = input[findlast(x->x=="size_rve_z",input)[1],3]
            end
            actualFiberVF = [actualFiberVolume/(sizeRVEx*sizeRVEy*sizeRVEz)]

            # Output result
            header = ["time" "e11" "e22" "e33" "2*e12" "2*e13" "2*e23" "s11" "s22" "s33" "s12" "s13" "s23" "a11" "a22" "a33" "a12" "a13" "a23" "vf"]
            result = [result repeat(actualOrientationTensor, size(result,1)) repeat(actualFiberVF, size(result,1))]
            open(outPath*stressStrainResultNameFormat*listOfSubmittedAnalysisNum[i]*".txt", "w") do io
                writedlm(io,header)
                writedlm(io,result)
            end
        else
            listOfIncompleteAnalysisNum = push!(listOfIncompleteAnalysisNum, listOfSubmittedAnalysisNum[i])
        end
        # Clear used variable
        unzippedMacFile = []
        unzippedInputFile = []
        unzippedOrientationFile = []
        unzippedFiberVolumeFile = []
        input = []
        result = []
        refFinalTime = []
        actualFinalTime = []
        actualFiberOrientation = []
        uniqueFiberNum = []
        theta = []
        phi = []
        p = []
        actualOrientationTensor = []
        actualFiberVolume = []
        sizeRVEx = []
        sizeRVEy = []
        sizeRVEz = []
        actualFiberVF = []
        
    end
    #listOfCompletedAnalysisNum = listOfSubmittedAnalysisNum
    #filter!(e->e ∉ listOfIncompleteAnalysisNum,listOfCompletedAnalysisNum)
    open(resultPath*"ListOfCompletedAnalysis.txt", "w") do io
        writedlm(io,["Total number of completed analysis: ";])
        writedlm(io,[string(length(listOfSubmittedAnalysisNum)-length(listOfIncompleteAnalysisNum));"";])
        writedlm(io,["Analysis completed time: ";])
        writedlm(io,[listOfSubmittedAnalysisNum listOfAnalysisActualFinalTime])
    end

    # Remove all file in inPath except the zip files
    rmFile = setdiff!(glob("**",inPath), glob("*.zip",inPath))
    for i = 1:length(rmFile)
        rm(rmFile[i])
    end
end

In [ ]:
# Main function
##----Parameters----############################################################
# Number of files is n_end - n_start
n_start = 1 # Start number
n_end = 20 # End number
# Ensure that counter is increasing!
if n_start > n_end
    n_start = 1
    n_end = 10
end

timeSteps = 100;
t = LinRange(0,1,timeSteps+1) # Time variable
outputPrecision = "5"

strainPath = "D:/Data_Generation/1D/StrainData/"
strainNameFormat = "strainData_"
orientationPath = "D:/Data_Generation/1D/OrientationData/"
orientationNameFormat = "orientationData_"
sizeRVENameFormat = "RVESize_"

dafPath = "D:/Data_Generation/1D/DigimatFile/"
dafNameFormat = "analysis_"
resultPath = "D:/Data_Generation/1D/RVEResult/"
outputFileNameFormat = "DefaultJobName_"
stressStrainResultNameFormat = "rveResult_"

finalTime = "1.0e+00"
maxTimeInc = string(1/timeSteps)
minTimeInc = string(1/timeSteps/10)
initialTimeInc = string(1/timeSteps)

youngFiber = "76e+09"
poissonFiber = "2.2e-01"
fiberVolumefraction = "1e-01" # With tensor orientation, this actual ref VF will be +0-50%
fiberDiameter = "1e-5"
fiberSize = "0.00024"
fiberOrientation = "tensor"

youngMatrix = "3.1e+09"
poissonMatrix = "3.5e-01"
yieldMatrix = "2.5e+07"
hardeningModulusMatrix = "2e+07"
hardeningExponentMatrix = "3.25e+02"
hardeningModulus2Matrix = "1.5e+08"
################################################################################


In [9]:
# IMPORTANT! Match significant digits of time vector to outputPrecision!
t = round.(t,sigdigits=parse(Int,outputPrecision))

# Generate strain path data. Optionally also generates orientation tensors.
for i = n_start:n_end
    noise = rand()
    n_drift = rand([1 2 5 10])
    # Calculate n_step by dividing timeStep by n_drift to ensure
    # constant length of time series
    n_step = div(timeSteps,n_drift) # div(x,y) to ensure int.
    EPS = 0.01 + rand()*0.04 # Maximum admissible component in 0.01 to 0.05
    if rand() > 0.9
        uni = true
    else
        uni = false
    end
    strainPathGenerator(noise,n_step,n_drift,EPS,t,strainPath*strainNameFormat*string(i),uni)
    udOrientationTensorGenerator(fiberVolumefraction,
    orientationPath*orientationNameFormat*string(i),
    orientationPath*sizeRVENameFormat*string(i))
end


# Write *.daf files for every strain path.
for i = n_start:n_end
    local inPath = strainPath*strainNameFormat*string(i)*".txt"
    local outPath = dafPath*dafNameFormat*string(i)*".daf"
    local inPath2 = orientationPath*orientationNameFormat*string(i)*".txt"
    local inPath3 = orientationPath*sizeRVENameFormat*string(i)*".txt"
    writedaf(inPath,inPath2,inPath3,outPath,finalTime,maxTimeInc,minTimeInc,initialTimeInc,
    youngFiber,poissonFiber,fiberDiameter,fiberSize,fiberOrientation,
    youngMatrix,poissonMatrix,yieldMatrix,hardeningModulusMatrix,hardeningExponentMatrix,hardeningModulus2Matrix)
end


In [ ]:
# Run DIGIMAT simulation for every *.daf file.
for i = n_start:n_end
    digimatPath = "C:/MSC.Software/Digimat/2022.4/DigimatFE/exec/"
    digimatInput = "input="*dafPath*dafNameFormat*string(i)*".daf"

    Shell.run(digimatPath*"DigimatFE.bat -runFEWorkflow "*digimatInput*" workingDir="*dafPath)

    # Delete unnecessary files to save space
    rmFile = setdiff!(glob("**",dafPath), [glob("*.daf",dafPath); glob("*.zip",dafPath); glob(dafNameFormat*"*.log",dafPath)])
    for i = 1:length(rmFile)
        rm(rmFile[i])
    end
end

In [4]:
# Extract result and calculate the actual orientation tensor, fiber volume fraction
extracrResult(dafPath,resultPath,dafNameFormat,outputFileNameFormat,stressStrainResultNameFormat)